In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
#from sklearn.metrics import f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.io import read_image, ImageReadMode
import torchvision
from torchvision.datasets.vision import VisionDataset
from tqdm.notebook import tqdm
from PIL import Image
from glob import glob
import os
import json

In [44]:
'''lstdir=os.listdir('/home/mar/Загрузки/benetech-making-graphs-accessible(1)/train/annotations')
for i in lstdir:
    s=""
    try:
            with open(i) as f:
                s=f.read()
    except:
            continue
    gen=json.loads(s)
    if len(gen['visual-elements']['boxplots'])!=0:
        print(i)
    del s'''

'lstdir=os.listdir(\'/home/mar/Загрузки/benetech-making-graphs-accessible(1)/train/annotations\')\nfor i in lstdir:\n    s=""\n    try:\n            with open(i) as f:\n                s=f.read()\n    except:\n            continue\n    gen=json.loads(s)\n    if len(gen[\'visual-elements\'][\'boxplots\'])!=0:\n        print(i)\n    del s'

In [45]:
def interesting(path,dimensions):
    def typization(string):
        if string=='chart_title':
            return (0,1,0)
        elif string=='axis_title':
            return (0,2,0)
        elif string=='tick_label':
            return (0,3,0)
        elif string=='plot-bb':
            return (0,4,0)
        elif string=='axes':
            return (0,5,0)
        elif string=="bars":
            return (0,6,0)
        elif string=="boxplots":
            return (0,7,0)
        elif string=="dot points":
            return (0,8,0)
        elif string=="lines":
            return (0,9,0)
        elif string=="scatter points":
            return (0,10,0)
#    creator = np.array(dimensions[1]*[dimensions[2]*[0]])
    creator = np.zeros((dimensions[1], dimensions[2]), dtype=np.uint8)
    creator = cv2.cvtColor(creator, cv2.COLOR_GRAY2BGR)
    print('img: ', dimensions,'\n')
    s=""
    with open(path) as f:
        s=f.read()
    gen=json.loads(s)
    gen2=gen['plot-bb']
    cv2.fillPoly(creator, [np.array([[(gen2['x0']+gen2['width'],gen2['y0'])],[(gen2['x0'],gen2['y0'])],[(gen2['x0'],gen2['y0']+gen2['height'])],[(gen2['x0']+gen2['width'],gen2['y0']+gen2['height'])]],dtype=np.int32)], typization('plot-bb'))
    for i in range(len(gen['text'])):
        if len(gen['text'][i]['polygon'])!=0:
            gen1=gen['text'][i]['polygon']
            cv2.fillPoly(creator, [np.array([[(gen1['x0'],gen1['y0'])],[(gen1['x1'],gen1['y1'])],[(gen1['x2'],gen1['y2'])],[(gen1['x3'],gen1['y3'])]],dtype=np.int32)], typization(gen['text'][i]['role']))
    axe=gen['axes']
    axex=axe['x-axis']['ticks']
    axey=axe['y-axis']['ticks']
    for i in range(len(axex)):
        cv2.fillPoly(creator, [np.array([[(axex[i]['tick_pt']['x'],axex[i]['tick_pt']['y'])],[(axex[i]['tick_pt']['x'],axex[i]['tick_pt']['y']-gen2['height'])]],dtype=np.int32)], typization('axes'))
    for i in range(len(axey)):
        cv2.fillPoly(creator, [np.array([[(axey[i]['tick_pt']['x'],axey[i]['tick_pt']['y'])],[(axey[i]['tick_pt']['x']+gen2['width'],axey[i]['tick_pt']['y'])]],dtype=np.int32)], typization('axes'))
    if len(gen["visual-elements"]["bars"])!=0:
        for el in gen["visual-elements"]["bars"]:
            cv2.fillPoly(creator, [np.array([[(el['x0']+el['width'],el['y0'])],[(el['x0'],el['y0'])],[(el['x0'],el['y0']+el['height'])],[(el['x0']+el['width'],el['y0']+el['height'])]],dtype=np.int32)], typization('bars'))
    elif len(gen["visual-elements"]["boxplots"])!=0:
        ...
    elif len(gen['visual-elements']['dot points'])!=0:
        for i in gen['visual-elements']['dot points'][0]:
            cv2.circle(creator, (int(i['x']), int(i['y'])), 4, typization('dot points'), 8)
    elif len(gen['visual-elements']['lines'])!=0:
        k=[]
        for i in gen['visual-elements']['lines'][0]:
            cv2.circle(creator, (int(i['x']), int(i['y'])), 2, typization('dot points'), 5)
        for i in range(1,len(gen['visual-elements']['lines'][0])):
            cv2.drawContours(creator,[np.array([(int(gen['visual-elements']['lines'][0][i-1]['x']),int(gen['visual-elements']['lines'][0][i-1]['y'])),(int(gen['visual-elements']['lines'][0][i]['x']),int(gen['visual-elements']['lines'][0][i]['y']))])],-1,typization('lines'),3)
    elif len(gen['visual-elements']['scatter points'])!=0:
        for i in gen['visual-elements']['scatter points'][0]:
            cv2.circle(creator, (int(i['x']), int(i['y'])), 2, typization('dot points'), 5)
    creator = cv2.cvtColor(creator, cv2.COLOR_BGR2GRAY)
    creator = np.array([creator]*3)
    output = torch.from_numpy(creator)
    print('mask: ',output.shape)
    return output
#    plt.imshow(creator)
#    plt.show()
#interesting('/home/mar/Загрузки/benetech-making-graphs-accessible(1)/train/annotations/0294808fcbc0.json',(1,600,600))

In [46]:
image_path = '/home/mar/Загрузки/benetech-making-graphs-accessible(1)/train/images'
mask_path = '/home/mar/Загрузки/benetech-making-graphs-accessible(1)/train/annotations'
labels = pd.read_csv('/home/mar/Загрузки/benetech-making-graphs-accessible(1)/sample_submission.csv')
print(labels.head())
classes = ['bars','boxplots',"dot points","lines","scatter points"]#labels.name.values.tolist()
print(classes)
length = len(os.listdir(image_path))

def img_transform(img, mask):

    img = img.float() / 255.0

    one_dim = img.shape[2]
    two_dim = img.shape[3]
    one_dim1 = mask.shape[2]
    two_dim1 = mask.shape[3]

    dim1 = torch.Tensor([[(4096-one_dim)*[[0]]]])
    dim2 = torch.Tensor([[[(4096-two_dim)*[0]]]])
    dim3 = torch.Tensor([[(4096-one_dim1)*[[0]]]])
    dim4 = torch.Tensor([[[(4096-two_dim1)*[0]]]])

    img = torch.cat((torch.cat((img,dim1),dim=2),dim2),dim=3)
    mask = torch.cat((torch.cat((mask,dim3),dim=2),dim4),dim=3)
    img = img.to(device)

    mask = mask.to(device)

    return img, mask.squeeze(1).long()

def img_transform1(img, mask):

    img = img.float() / 255.0

    one_dim = img.shape[1]
    two_dim = img.shape[2]
    one_dim1 = mask.shape[1]
    two_dim1 = mask.shape[2]

    dim1 = torch.Tensor([(4096-one_dim)*[[0]]])
    dim2 = torch.Tensor([[(4096-two_dim)*[0]]])
    dim3 = torch.Tensor([(4096-one_dim1)*[[0]]])
    dim4 = torch.Tensor([[(4096-two_dim1)*[0]]])

    img = torch.cat((torch.cat((img,dim1),dim=1),dim2),dim=2)
    mask = torch.cat((torch.cat((mask,dim3),dim=1),dim4),dim=2)
    img = img.to(device)

    mask = mask.to(device)

    return img, mask#.squeeze(1).long()

class Graphics(Dataset):
    def __init__(self, imgs_dir, masks_dir, count, is_val = False):
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir

        imgs_paths = os.listdir(self.imgs_dir)
        imgs_paths.sort()

        mask_paths = os.listdir(self.masks_dir)
        mask_paths.sort()

        self.is_val = is_val
        if not is_val:  # для разделения на train/val в процессе
            self.imgs_paths = imgs_paths[:count]
            self.mask_paths = mask_paths[:count]
        else:
            self.imgs_paths = imgs_paths[-count:]
            self.mask_paths = mask_paths[-count:]

    def __len__(self):
        return len(self.imgs_paths)

    def __getitem__(self, idx):
        img = read_image(os.path.join(self.imgs_dir, self.imgs_paths[idx]), ImageReadMode.RGB)
        mask = interesting(os.path.join(self.masks_dir, self.mask_paths[idx]),img.shape)
#        mask = read_image(os.path.join(self.masks_dir, self.mask_paths[idx]), ImageReadMode.GRAY)

        return img, mask#img_transform1(img, mask)
    
torchvision.models.segmentation.DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1.transforms()


train_dataset_len = int(length * 0.7)
val_dataset_len = length - train_dataset_len

train_dataset = Graphics(image_path, mask_path, train_dataset_len)

val_dataset = Graphics(image_path, mask_path, val_dataset_len, is_val=True)

print(train_dataset[5][0].shape)
print(train_dataset[5][1].shape)
img, mask = next(iter(train_dataset))

batch_size = 2

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2)

val_loader = DataLoader(val_dataset, batch_size, shuffle=False, num_workers=2)

model = torchvision.models.segmentation.deeplabv3_resnet50(weights=torchvision.models.segmentation.DeepLabV3_ResNet50_Weights.DEFAULT,
                                                    progress=True)

               id data_series    chart_type
0  000b92c3b098_x     abc;def  vertical_bar
1  000b92c3b098_y     0.0;1.0  vertical_bar
2  007a18eb4e09_x     abc;def  vertical_bar
3  007a18eb4e09_y     0.0;1.0  vertical_bar
4  00dcf883a459_x     abc;def  vertical_bar
['bars', 'boxplots', 'dot points', 'lines', 'scatter points']
img:  torch.Size([3, 287, 497]) 

mask:  torch.Size([3, 287, 497])
torch.Size([3, 287, 497])
img:  torch.Size([3, 287, 497]) 

mask:  torch.Size([3, 287, 497])
torch.Size([3, 287, 497])
img:  torch.Size([3, 281, 466]) 

mask:  torch.Size([3, 281, 466])


In [47]:
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision.models.segmentation.fcn import FCNHead
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
model.classifier = DeepLabHead(2048, 23)
model.aux_classifier = FCNHead(1024, 23)
model = model.to(device)

from torch.nn import CrossEntropyLoss
import torch.nn.functional as F

loss = CrossEntropyLoss().to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def pixel_accuracy(mask, output):
    output_softmax = F.softmax(output, dim=1)
    output_argmax = torch.argmax(output_softmax, dim=1)
#softmax(zi) = exp(zi)/sum[k:1..K](exp(zk))
#Свойства softmaxя
    bool_tensor = (torch.flatten(mask)) == (torch.flatten(output_argmax))

    return torch.sum(bool_tensor) / torch.numel(bool_tensor)

In [ ]:
from tqdm import tqdm
epoch_count = 30

train_losses = []
val_losses = []

train_accs = []
val_accs = []
es_steps = 3
count_steps = 0

train_len = len(train_loader)
val_len = len(val_loader)
print(train_len)
print(val_len)

best_score = 1e10

for epoch in range(epoch_count):
    if count_steps >= es_steps:
        print("Early stopping!")
        break

    train_loss_sum = 0
    train_pixel_acc = 0

    model.train()
    for img_batch, mask_batch in tqdm(train_loader):
#        print(epoch)
        img_batch = img_batch.to(device, non_blocking=True)
        mask_batch = mask_batch.to(device, non_blocking=True)
        img_batch, mask_batch = img_transform(img_batch, mask_batch, is_val=False)

        optimizer.zero_grad()

        output_batch = model(img_batch)
        loss_value = loss(output_batch['out'], mask_batch)

        train_pixel_acc += pixel_accuracy(mask_batch, output_batch['out']).detach() #Обучить нейросеть с использованием pixel_accuracy в качестве loss-функции
        train_loss_sum += loss_value.detach()

        loss_value.backward()
        optimizer.step()
        
        del output_batch

    train_loss = train_loss_sum / train_len
    train_acc = train_pixel_acc / train_len

    train_losses.append(train_loss)
    train_accs.append(train_acc)

    print(f"Epoch {epoch} / {epoch_count} | train loss = {train_loss} | train acc = {train_acc}")

    model.eval()

    val_loss_sum = 0
    val_pixel_acc = 0

    for img_batch, mask_batch in (val_loader):
        img_batch = img_batch.to(device, non_blocking=True)
        mask_batch = mask_batch.to(device, non_blocking=True)
        img_batch, mask_batch = img_transform(img_batch, mask_batch, is_val=True)

        output_batch = model(img_batch)
        loss_value = loss(output_batch['out'], mask_batch)

        val_loss_sum = val_loss_sum + loss_value.detach()
        val_pixel_acc = val_pixel_acc + pixel_accuracy(mask_batch, output_batch['out']).detach()
        
        del output_batch

    val_loss = val_loss_sum / val_len
    val_acc = val_pixel_acc / val_len

    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f"Epoch {epoch} / {epoch_count} | val loss = {val_loss} | val acc = {val_acc}")

    if val_loss < best_score:
        best_score = val_loss
        count_steps = 0
        torch.save(model, "best_model.pt")
    else:
        count_steps += 1

21202
9087


  0%|          | 0/21202 [00:00<?, ?it/s]

img:  torch.Size([3, 272, 469]) 

mask:  torch.Size([3, 272, 469])
img:  img: torch.Size([3, 276, 464])  
torch.Size([3, 268, 464])
 

mask:  mask: torch.Size([3, 276, 464]) 
torch.Size([3, 268, 464])
img:  torch.Size([3, 290, 490]) 

mask:  torch.Size([3, 290, 490])


  0%|          | 0/21202 [00:00<?, ?it/s]

img:  torch.Size([3, 283, 504]) 

img: 

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/mar/.local/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/mar/.local/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/mar/.local/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 264, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mar/.local/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mask:   torch.Size([3, 283, 504])torch.Size([3, 347, 487]) 


mask: img:  

 torch.Size([3, 347, 487])torch.Size([3, 328, 501])
 

mask: img:   torch.Size([3, 328, 501])torch.Size([3, 299, 509])
 

mask:  torch.Size([3, 299, 509])
img:  torch.Size([3, 275, 466]) 

mask:  torch.Size([3, 275, 466])
img:  torch.Size([3, 480, 640]) 

mask:  torch.Size([3, 480, 640])


In [ ]:
import matplotlib.pyplot as plt
train_losses = [x.cpu().item() for x in train_losses]
val_losses = [x.cpu().item() for x in val_losses]
plt.plot(train_losses, linestyle="-")
plt.plot(val_losses, linestyle="--")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
train_accs = [x.cpu().item() for x in train_accs]
val_accs = [x.cpu().item() for x in val_accs]

plt.plot(train_accs, linestyle="-")
plt.plot(val_accs, linestyle="--")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

In [ ]:
model.eval()
label_map = np.array([
    (0, 0, 0),  # unlabeled
    (128, 64, 128),  # chart_title
    (130, 76, 0),  # axis_title
    (0, 102, 0),  # tick_label
    (112, 103, 87),  # plot-bb
    (28, 42, 168),  # axes
    (48, 41, 30),  # bars
    (0, 50, 89), # boxplots
    (107, 142, 35),  # dot points
    (70, 70, 70),  # lines
    (102, 102, 156),  # scatter points
    (255, 0, 0), # conflicting
])

def draw_segmentation_map(outputs):
    labels = torch.argmax(outputs.squeeze(), dim=0).numpy()
  
    # Create 3 Numpy arrays containing zeros.
    # Later each pixel will be filled with respective red, green, and blue pixels
    # depending on the predicted class.
  
    red_map   = np.zeros_like(labels).astype(np.uint8)
    green_map = np.zeros_like(labels).astype(np.uint8)
    blue_map  = np.zeros_like(labels).astype(np.uint8)
  
    for label_num in range(0, len(label_map)):
        index = labels == label_num
         
        R, G, B = label_map[label_num]
  
        red_map[index]   = R
        green_map[index] = G
        blue_map[index]  = B
  
    segmentation_map = np.stack([red_map, green_map, blue_map], axis=2)
    return segmentation_map

def image_overlay(image, segmented_image):
    alpha = 1  # transparency for the original image
    beta  = 0.8  # transparency for the segmentation map
    gamma = 0  # scalar added to each sum
  
    image = np.array(image)
    segmented_image = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
     
    cv2.addWeighted(image, alpha, segmented_image, beta, gamma, image)
  
    return image

imgs_paths = os.listdir(image_path)
imgs_paths.sort()

def perform_inference(model=model ,imgs_paths=imgs_paths, num_images=10, image_dir='/home/mar/Загрузки/benetech-making-graphs-accessible(1)/train/images/', device='cpu'):
     
     
    device = device if device is not None else ("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    #preprocess = transforms.Compose([
    #  transforms.Resize([520, 520]),
    #  transforms.ToTensor(),
    #  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #])
    # Load image handles for the validation set.
  
    # Randomly select 'num_images' from the whole set for inference.
    selected_images = np.random.choice(imgs_paths, num_images, replace=False)
  
    # Iterate over selected images
    for img_name in selected_images:
         
        # Load and pre-process image.
        image_path = os.path.join(image_dir, img_name)
        img_raw = Image.open(image_path).convert("RGB")
        W, H = img_raw.size[:2]
        img_t = img_transform(img_raw)
        img_t = torch.unsqueeze(img_t, dim=0).to(device)
  
        # Model Inference
        with torch.no_grad():
            output = model(img_t)["out"].cpu()
  
        # Get RGB segmentation map
        segmented_image = draw_segmentation_map(output)
  
        # Resize to original image size
        segmented_image = cv2.resize(segmented_image, (W, H), cv2.INTER_LINEAR)
        overlayed_image = image_overlay(img_raw, segmented_image)
         
        # Plot
        plt.figure(figsize=(12, 10), dpi=100)
        plt.subplot(1, 3, 1)
        plt.axis("off")
        plt.title("Image")
        plt.imshow(np.asarray(img_raw))
  
        plt.subplot(1, 3, 2)
        plt.title("Segmentation")
        plt.axis("off")
        plt.imshow(segmented_image)
  
        plt.subplot(1, 3, 3)
        plt.title("Overlayed")
        plt.axis("off")
        plt.imshow(overlayed_image[:, :, ::-1])
         
        plt.show()
        plt.close()

  
    return

perform_inference()